<a href="https://colab.research.google.com/github/AaronAbraham420/NeuralAudioStemSeparation/blob/master/NeuralAudioStemSeparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Audio Stem Separation

Colab notebook for [Deezer Spleeter](https://github.com/deezer/spleeter) and [Facebook Research Demucs](https://github.com/facebookresearch/demucs), to be run in Google Colab, using Google Drive as data storage.

`input` value may be
1.   Path to an individual audio file located in your Google Drive.
2.   Path to a directory in your Google Drive containing multiple audio files. Each audio file found in the directory will be individually processed.
3.   Single YouTube URL
4.   Multiple YouTube URLs separated by spaces. Each youtube video will be individually processed.

Input will be separated to 4 stems: vocals, drums, bass and other. Stems will be saved as WAV files to your Google Drive. Enter `output_dir` relative to your Google Drive root.

In [ ]:
#@title #Setup
#@markdown This cell needs to be run only once. It will mount your Google Drive and setup prerequisites.<br>
#@markdown <small>Mounting Drive will enable this notebook to save outputs directly to your Drive. Otherwise you will need to copy/download them manually from this notebook.</small>

force_setup = False
pip_packages = 'spleeter demucs youtube-dl'
main_repository = ''
mount_drive = True #@param {type:"boolean"}

# Download the repo from Github
import os
from google.colab import output
import warnings
warnings.filterwarnings('ignore')
%cd /content/

# inhagcutils
if not os.path.isfile('/content/inhagcutils.ipynb') and force_setup == False:
  !apt -q install ffmpeg
  !pip -q install import-ipynb {pip_packages}
  !curl -s -O https://raw.githubusercontent.com/olaviinha/inhagcutils/master/inhagcutils.ipynb
import import_ipynb
from inhagcutils import *

# Mount Drive
if mount_drive is True:
  if not os.path.isdir('/content/drive'):
    from google.colab import drive
    drive.mount('/content/drive')
    drive_root = '/content/drive/My Drive'
  if not os.path.isdir('/content/mydrive'):
    os.symlink('/content/drive/My Drive', '/content/mydrive')
    drive_root = '/content/mydrive/'
  drive_root_set = True
else:
  create_dirs(['/content/faux_drive'])
  drive_root = '/content/faux_drive/'

!gsutil -q -m cp -R gs://neural-research/olaviinha/spleeter-configs/custom-4stems-22kHz-z.json /content/cfg.json

if main_repository is not '':
  !git clone {main_repository}

output.clear()
# !nvidia-smi
op(c.ok, 'Setup finished.')

<>:41: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:41: SyntaxWarning: "is not" with a literal. Did you mean "!="?


/content
Reading package lists...
Building dependency tree...
Reading state information...
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.3/77.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.8/511.8 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.1/87.1 kB 12.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#@title Separate

input = "" #@param {type:"string"}
separate_by = "Demucs" #@param ["Spleeter", "Demucs"] {type:"string"}
output_dir = "" #@param {type:"string"}

#----------------------------------------------------------------------------#

import os
from os.path import isfile, join
from glob import glob
import time, datetime

input_type = check_input_type(input)
tmp_dir = '/content/tmp/'
seps_dir = fix_path(drive_root+output_dir)
#seps_dir = '/content/tmp/seps'

start_point = time.time()

if not os.path.isdir(tmp_dir):
  create_dirs([tmp_dir])
# if not os.path.isdir(output_dir):
#   create_dirs([seps_dir])

if output_dir == '' or output_dir == 'same':
  output_dir = fix_path(drive_root+path_dir(input))
else:
  output_dir = fix_path(drive_root+output_dir)
  if not os.path.isdir(output_dir):
    create_dirs([output_dir])

if input_type == 'unknown':
  input = drive_root+input
  input_type = check_input_type(input)
  if input_type=='dir':
    input = fix_path(input)

if input_type == 'youtube':
  !pip -q install youtube-dl
  !youtube-dl --restrict-filenames -x --no-continue --audio-format wav -o "{tmp_dir}%(title)s.%(ext)s" {input}

from spleeter.separator import Separator
separator = Separator('/content/cfg.json')

if input_type == "dir" or input_type == "youtube":
  if input_type == "dir":
    target = input
  if input_type == "youtube":
    target = dir_tmp
  file_list = list_audio(target)
  for audiofile in file_list:
    op(c.title, '\nSeparate '+audiofile+' to '+seps_dir.replace(drive_root, '')+basename(input))
    # Spleeter Python API is outdated for Colab
    # separator.separate_to_file(audiofile, seps_dir)
    if separate_by is "Spleeter":
      !spleeter separate -p /content/cfg.json -o "{output_dir}" "{audiofile}"
    if separate_by is "Demucs":
      %cd "{output_dir}"
      !demucs "{audiofile}"
    op(c.ok, 'FIN.', 'Files saved under directory '+output_dir.replace(drive_root, '')+basename(audiofile)+'/')
elif input_type == "file":
  # Spleeter Python API is outdated for Colab
  # separator.separate_to_file(input, seps_dir)
  if separate_by is "Spleeter":
    !spleeter separate -p /content/cfg.json -o "{output_dir}" "{input}"
  if separate_by is "Demucs":
    %cd "{output_dir}"
    !demucs "{input}"

  elapsed = round(time.time()-start_point)
  # elapsed_hr = strftime("%H:%M:%S", gmtime(elapsed))
  print('Elapsed', elapsed, 'seconds.')
  op(c.ok, 'FIN.', 'Files saved under directory '+output_dir.replace(drive_root, '')+basename(input)+'/')
else:
  print("We're not sure what you want to separate. Please check the input field.")